In [1]:
import numpy as np
import strawberryfields as sf
import strawberryfields.ops as ops

In [2]:
nbar = 6.22 # mean photon number
v = 2*nbar+1 # v=2, .5 mean photon
#ν := 2n̄ + 1
tmsv_cov = np.array([
    [v,np.sqrt(v**2-1),0,0],
    [np.sqrt(v**2-1),v,0,0],
    [0,0,v,-np.sqrt(v**2-1)],
    [0,0,-np.sqrt(v**2-1),v],
])

In [3]:
# https://strawberryfields.readthedocs.io/en/stable/code/api/strawberryfields.ops.BSgate.html
transmitivity = .52
theta = np.arccos(np.sqrt(transmitivity)) # transmission amplitude
phi = 0#np.pi # phase angle; np.pi/2 is the symmetric beam splitter

In [4]:
# params for state preparations

t_nbar = np.random.uniform(0,12)
d_r = np.random.uniform(-10,10)
d_phi = np.random.uniform(-np.pi,np.pi)

print(t_nbar,d_r,d_phi)

7.315227748347578 8.895040938295388 1.1212036097319187


In [5]:
exp = sf.Program(3) # 1 for the mode goinging through the channel, 2 for the explicit environment

with exp.context as q:
    # q[2] is the mode going through our custom thermal loss channel
    
    # Prepare q[2] in some state
    ops.Thermal(t_nbar) | q[2]
    ops.Dgate(d_r,d_phi) | q[2]
    
    # Prepare the environment
    ops.Gaussian(tmsv_cov,np.zeros(4)) | [q[0],q[1]] # prepares a gaussian state
    ops.BSgate(theta, phi) | (q[1], q[2]) # applies BS
    
    # We will compare our results against the built in Thermal loss channel
    #ops.ThermalLossChannel(transmitivity,nbar) | q[2]

In [6]:
eng = sf.Engine(backend="gaussian")
#eng = sf.Engine(backend="tf", backend_options={"cutoff_dim": 10})

custom_results = eng.run(exp)

In [7]:
exp = sf.Program(1) # we just need one mode since we don't have an explicit environment

with exp.context as q:
    # q[0] is the mode going through the builtin  thermal loss channel
    
    # Prepare q[2] in some state
    ops.Thermal(t_nbar)| q[0]
    ops.Dgate(d_r,d_phi) | q[0]

    # We will compare our results against the built in Thermal loss channel
    ops.ThermalLossChannel(transmitivity,nbar) | q[0]

In [8]:
eng = sf.Engine(backend="gaussian")
#eng = sf.Engine(backend="tf", backend_options={"cutoff_dim": 10})

builtin_results = eng.run(exp)

In [9]:
def extract_state_cov(cov,state_num):
    num_modes = len(cov)//2
    new_cov = np.zeros((2,2))
    new_cov[0,0] = cov[state_num,state_num]
    new_cov[1,1] = cov[state_num+num_modes,state_num+num_modes]
    new_cov[1,0] = cov[state_num+num_modes,state_num]
    new_cov[0,1] = cov[state_num,state_num+num_modes]
    return new_cov

def extract_state_means(means,state_num):
    num_modes = len(means)//2
    new_means = np.zeros(2)
    new_means[0] = means[state_num]
    new_means[1] = means[state_num+num_modes]
    return new_means


In [10]:
print(extract_state_cov(custom_results.state.cov(),2))
print(extract_state_means(custom_results.state.means(),2))

[[1.45790369e+01 3.30009844e-16]
 [3.30009844e-16 1.45790369e+01]]
[ 5.57529822 11.55375682]


In [11]:
print(builtin_results.state.cov())
print(builtin_results.state.means())

[[14.57903686  0.        ]
 [ 0.         14.57903686]]
[ 5.57529822 11.55375682]
